In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
from langchain_openai import OpenAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import create_react_agent
from langchain import hub
prompt = hub.pull('hwchase17/react')

llm = OpenAI()
search = SerpAPIWrapper()
tools = [
    Tool(
        name = 'Search',
        func = search.run,
        description = 'LLM이 관련 지식이 없을 때 지식 검색에 사용됩니다.'
    ),
]
agent = create_react_agent(llm,tools,prompt)

# agentexecutor 가져오기
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose=True)
print('첫 번째 실행결과:')
agent_executor.invoke({'input':'현재 인공지능 에이전트의 최신 연구동향은 무엇인가?'})

첫 번째 실행결과:


> Entering new AgentExecutor chain...
 최신 연구동향에 대해 궁금하면, 검색을 통해 알아보는 것이 유용할 것 같다.
Action: Search
Action Input: "최신 인공지능 연구"['이 글에서는 AI타임스, 인공지능신문부터 MIT Technology Review, DeepMind Blog까지 국내외 주요 AI 정보 허브를 소개합니다. AI 기술 동향, 산업 분석, ...', '2025년에 주목해야 할 10가지 인공 지능 트렌드 · 1) AI 에이전트 · 2) 초개인화 · 3) AI ROI 측정 · 4) 생성형 AI 보안 제품 · 5) 양자 AI · 6) 대화형 AI · 7) ...', '인공지능 분야의 최신 연구 동향은 현재 기술이 나아가고 있는 방향과 발전 속도를 이해하는 데 핵심적인 역할을 합니다. 이 보고서에서는 인공지능의 ...', '1. 제 목 : 2024년 국내외 인공지능 산업 동향 연구 · 2. 연구 배경 및 목적; ㅇ 생성AI 기술의 부상으로 인공지능 산업 생태계가 빠르게 변화 · 3. 연구의 ...', '최근에는 다양한 데이터 유형을 동시에 이해하고 통합 처리하는 멀티모달 AI가 속속 등장하고 있습니다. 일례로 오픈 AI의 GPT-4는 텍스트뿐만 아니라 ...', "Google에서 혁신적인 Machine Learning(ML) 모델인 'Google Mind'를 발표했습니다. 이 모델은 인간의 뇌를 닮은 구조와 기능을 가지고 있으며, 인공 지능 ...", '일정한 수준에서 AI는 자체 알고리즘을 개발하여 문제를 해결하고, 효율성을 증대하고 인간에게 유용한 연구 데이터를 제공할 수 있습니다. 자동화된 AI를 사용하면 ...', '토픽 모델링으로 살펴본 인공지능(AI) 논문의 연구 주제 살펴보기 · 인공지능(AI) 관련 논문에서 사용된 키워드 간의 연결 관계 살펴보기 · AI 어시스턴트로 ...', '최신 인공지능 기술동향. State of AI 2022 번역 오프라인 강화학습 기술동향

{'input': '현재 인공지능 에이전트의 최신 연구동향은 무엇인가?',
 'output': '현재 인공지능 에이전트의 최신 연구동향은 다양한 분야에서 발전하고 있으며, 새로운 소식들도 매우 흥미로운 것으로 보인다.'}

## ReAct 기반 체계 기반의 프롬프트
### 모델의 출력 품질은 제공된 입력 정보의 품질 및 구조와 밀접하게 관련되어 있다. 먼저 이 코드가 어떻게 프롬프트를 가져오는지 살펴본다. 

In [14]:
# LangChain Hub 가져오기
from langchain import hub
from pprint import pprint
# LangChain Hub에서 ReAct의 프롬프트 가져오기
prompt = hub.pull('hwchase17/react')
prompt.pretty_print()

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### LLM 인스턴스 생성
- 에이전트를 구축하려면 먼저 에이전트의 추론 엔진 역할을 하는 LLM의 인스턴스를 생성

In [17]:
#OpenAI 가져오기
from langchain_openai import OpenAI
llm = OpenAI()

### 검색 도구 정의
- 앞에서 LLM 인스턴스를 설정, 검색 도구 한가지만 사용하도록 구성

In [18]:
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
#SerpAPIWrapper 인스턴스화
search = SerpAPIWrapper()
tools = [
    Tool(
        name = 'Search',
        func = search.run,
        description = 'LLM이 관련 지시깅 없을 때 지식 검색에 사용됩니다.'
    ),
]
#위의 코드에서 SerpAPIWrapper는 SerpApi와의 상호작용을 감싸는 wrapepr

In [19]:
from langchain.agents import create_react_agent

agent = create_react_agent(llm,tools,prompt)